A recuperação é o ponto central do nosso fluxo de geração aumentada por recuperação (RAG).

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

Vamos acessar o nosso vectorDB de antes.

In [2]:
persist_directory = 'D:\Github\Chat-with-your-docs\Vectorstores-and-Embeddings\docs\chroma'

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\vinic\AppData\Local\Temp\ipykernel_17400\453241271.py:1: SyntaxWarning: invalid escape sequence '\G'
  persist_directory = 'D:\Github\Chat-with-your-docs\Vectorstores-and-Embeddings\docs\chroma'


In [3]:
# # Para que possui chava OpenAI
# embedding = OpenAIEmbeddings()
# vectordb = Chroma(
#     persist_directory=persist_directory,
#     embedding_function=embedding
# )

In [4]:
# Utilizando embeddings Huggingface
embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

#print resultados
print(vectordb._collection.count())

C:\Users\vinic\AppData\Local\Temp\ipykernel_17400\2937529488.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
d:\Github\Chat-with-your-docs\conda-gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\vinic\AppData\Local\Temp\ipykernel_17400\2937529488.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version 

209


Aqui será mostrado como é calculado a similaridade entre a pergunta e os textos abaixo

In [5]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [6]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [7]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [8]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [9]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

# Addressing Diversity: Maximum marginal relevance¶
Last class we introduced one problem: how to enforce diversity in the search results.

Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results.

In [10]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [11]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [12]:
docs_ss[1].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

Note the difference in results with MMR.

In [13]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [14]:
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [15]:
docs_mmr[1].page_content[:100]

"Okay, and using this matrix vector notati on, I think, I don't know, I think we did this \nwhole thin"

# Addressing Specificity: working with metadata¶
In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on metadata.

metadata provides context for each embedded chunk.

In [16]:
question = "what did they say about regression in the third lecture?"

In [17]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source": "D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf"}
)

for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf'}
{'page': 11, 'source': 'D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf'}
{'page': 13, 'source': 'D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf'}


# Addressing Specificity: working with metadata using self-query retriever
But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use SelfQueryRetriever, which uses an LLM to extract:

The query string to use for vector search
A metadata filter to pass in as well
Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [24]:
# from langchain.llms import OpenAI # Para quem possui chave da OpenAI
# from langchain_ollama import OllamaLLM
from langchain_community.llms import Ollama
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [45]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture01.pdf`, `D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture02.pdf`, or `D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

Note: The default model for OpenAI ("from langchain.llms import OpenAI") is text-davinci-003. Due to the deprication of OpenAI's model text-davinci-003 on 4 January 2024, you'll be using OpenAI's recommended replacement model gpt-3.5-turbo-instruct instead.

In [46]:
document_content_description = "Lecture notes"
llm = Ollama(model='gemma2:2b', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [47]:
question = "what did they say about regression in the third lecture?"

You will receive a warning about predict_and_parse being deprecated the first time you executing the next line. This can be safely ignored.

In [48]:
docs = retriever.get_relevant_documents(question)

In [49]:
for d in docs:
    print(d.metadata)

{'page': 10, 'source': 'D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf'}
{'page': 4, 'source': 'D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf'}
{'page': 10, 'source': 'D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture03.pdf'}


# Additional tricks: compression
Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this.

In [50]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [51]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [52]:
# Wrap our vectorstore
llm = Ollama(model='gemma2:2b', temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [53]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [54]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

MATLAB is a programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to  learn tool to use for implementing a lot of learning algorithms. 
And in case some of you want to work on your  own home computer or something if you don't have a MATLAB license, for the purposes of  this class, there's also — [inaudible] write that down [inaudible] MATLAB — there' s also a software package called Octave 
that you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it's free, and for the purposes of  this class, it will work for just about 
everything. 
So actually I, well, so yeah, just a side comment for those of you that haven't seen 
MATLAB before I guess, once a colleague of mine at a different university, not at 
Stanford, actually teaches another machine l earning course. He's taught it for many years.
-----------

# Combining various techniques

In [55]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [56]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

MATLAB is a programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to  learn tool to use for implementing a lot of learning algorithms. 
And in case some of you want to work on your  own home computer or something if you don't have a MATLAB license, for the purposes of  this class, there's also — [inaudible] write that down [inaudible] MATLAB — there' s also a software package called Octave 
that you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it's free, and for the purposes of  this class, it will work for just about 
everything. 
So actually I, well, so yeah, just a side comment for those of you that haven't seen 
MATLAB before I guess, once a colleague of mine at a different university, not at 
Stanford, actually teaches another machine l earning course. He's taught it for many years.
-----------

# Other types of retrieval¶
It's worth noting that vectordb as not the only kind of tool to retrieve documents.

The LangChain retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [57]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [59]:
# Load PDF
loader = PyPDFLoader("D:/Github/Chat-with-your-docs/Load-docs/docs/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [60]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [61]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

Document(metadata={}, page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes")

In [62]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(metadata={}, page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is 